## 작업형2 - 통신사에서 고객에게 청구될 총 금액을 예측하시오.
- 제공된 데이터 목록:
    - churn_train.csv(훈련 데이터)
    - churn_test.csv(평가용 데이터)
- 예측할 컬럼: TotalCharges(총 청구액)
- 학습용 데이터(churn_train.csv)를 이용하여 총 청구액을 예측하는 모델을 만든 후 이를 평가용 데이터(churn_test.csv)
에 적용해 얻은 예측값을 다음과 같은 형식의 CSV 파일로 생성하시오.

- 제출 파일은 다음 1개의 컬럼을 포함해야 한다.
    - pred: 예측된 총 청구액
    - 제출 파일명: ‘result.csv’
- 제출한 모델의 성능은 MAE(Mean Absolute Error) 평가지표에 따라 채점한다.


## 문제정의

In [63]:
# 문제정의
# 평가: MAE
# target: TotalCharges
# 최종파일: result.csv(컬럼 1개 pred)

## 데이터 불러오기

In [64]:
# 라이브러리 및 데이터 불러오기
import pandas as pd
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/8_2/churn_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/8_2/churn_test.csv")

## 탐색적 데이터 분석(EDA)

In [65]:
# 데이터 샘플
train.head()



,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges
0,CUST0454,Male,0,No,No,7,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Electronic check,163.45
1,CUST1145,Female,1,No,Yes,53,No,No phone service,Fiber optic,Yes,Yes,No,Yes,Yes,Yes,Month-to-month,No,Credit card,4140.89
2,CUST1138,Female,1,No,Yes,68,No,No phone service,DSL,Yes,Yes,Yes,No,No,Yes,Month-to-month,No,Mailed check,5152.36
3,CUST2645,Male,1,No,Yes,44,No,No phone service,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,Yes,Credit card,5251.84
4,CUST2632,Male,0,Yes,No,7,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,No,One year,No,Electronic check,725.69


In [66]:
# 데이터 크기
print(train.shape , test.shape)


(4116, 19) (1764, 18)


In [67]:
# 데이터 정보(자료형)
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4116 entries, 0 to 4115
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        4116 non-null   object 
 1   gender            4116 non-null   object 
 2   SeniorCitizen     4116 non-null   int64  
 3   Partner           4116 non-null   object 
 4   Dependents        4116 non-null   object 
 5   tenure            4116 non-null   int64  
 6   PhoneService      4116 non-null   object 
 7   MultipleLines     4116 non-null   object 
 8   InternetService   4116 non-null   object 
 9   OnlineSecurity    4116 non-null   object 
 10  OnlineBackup      4116 non-null   object 
 11  DeviceProtection  4116 non-null   object 
 12  TechSupport       4116 non-null   object 
 13  StreamingTV       4116 non-null   object 
 14  StreamingMovies   4116 non-null   object 
 15  Contract          4116 non-null   object 
 16  PaperlessBilling  4116 non-null   object 


In [68]:
# train 결측치 수
train.select_dtypes(include='object')

,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod
0,CUST0454,Male,No,No,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Electronic check
1,CUST1145,Female,No,Yes,No,No phone service,Fiber optic,Yes,Yes,No,Yes,Yes,Yes,Month-to-month,No,Credit card
2,CUST1138,Female,No,Yes,No,No phone service,DSL,Yes,Yes,Yes,No,No,Yes,Month-to-month,No,Mailed check
3,CUST2645,Male,No,Yes,No,No phone service,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,Yes,Credit card
4,CUST2632,Male,Yes,No,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,No,One year,No,Electronic check
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4111,CUST5809,Female,Yes,Yes,No,No phone service,DSL,No,No,Yes,Yes,No,Yes,Month-to-month,Yes,Credit card
4112,CUST4720,Male,Yes,Yes,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Mailed check
4113,CUST0173,Female,Yes,Yes,No,No phone service,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,Yes,Credit card
4114,CUST1244,Male,No,No,No,No phone service,DSL,Yes,Yes,Yes,No,Yes,Yes,One year,Yes,Bank transfer


In [69]:
# test 결측치 수




In [70]:
# customerID unique 수
train['customerID'].value_counts().sum()


4116

In [71]:
# target 기술 통계
# 회귀같은 경우는 describe가 아니라 value_counts를 쓰고 분류는 describe로 봐라!!
print(train['TotalCharges'].describe())

count    4116.000000
mean     2566.580782
std      1911.356766
min        20.030000
25%      1020.922500
50%      2144.285000
75%      3765.670000
max      8589.600000
Name: TotalCharges, dtype: float64


## 데이터 전처리

In [72]:
# 데이터 전처리
#object만 따로 뺼까? 어떻게 할까
# custmoer ID drop 하고 , target값은 pop 하고 , test custmer Id 도 drop 하고
train = train.drop('customerID', axis=1)
target = train.pop('TotalCharges')
test= test.drop('customerID', axis=1)


In [78]:
# 레이블 인코딩
# 이거 trian[col] 요런걸로 했었는데 기억이안남
# 근데 이거 object 형식만 인코딩 할거라서 col로 넣긴해야함

from sklearn.preprocessing import LabelEncoder
cols = train.select_dtypes(include='object').columns
for col in cols:
  le = LabelEncoder()
  train[col] = le.fit_transform(train[col])
  test[col] = le.transform(test[col])



In [79]:
train.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod
0,1,0,0,0,7,1,2,2,1,1,1,1,1,1,2,0,2
1,0,1,0,1,53,0,1,1,2,2,0,2,2,2,0,0,1
2,0,1,0,1,68,0,1,0,2,2,2,0,0,2,0,0,3
3,1,1,0,1,44,0,1,2,1,1,1,1,1,1,0,1,1
4,1,0,1,0,7,1,2,1,0,2,2,0,2,0,1,0,2


In [80]:
# 인코딩 후 데이터 샘플 확인
print(train.shape, test.shape)



(4116, 17) (1764, 17)


## 검증데이터 나누기

In [81]:
# 검증데이터 분리
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train,
                                            target,
                                            test_size=0.2,
                                            random_state=0)





In [82]:
print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)

(3292, 17) (824, 17) (3292,) (824,)


## 모델 학습 및 평가

In [83]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_tr,y_tr)
pred = rf.predict(X_val)
pred[:10]


array([3349.7385, 3867.7364, 3725.1204, 3614.047 , 3218.2395, 2273.4705,
       3659.4007, 2070.3966, 3208.7487, 3754.8443])

In [84]:
# MAE
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(y_val, pred))
# 958.7569927669903

#이렇게 계산했을때 이게 기술통계 train['TotalCharges'].describe() 해서min , 25%, 50%, 75%, max 값과 얼추 비슷해야함! 너무 동떨어지면 안됨
# 내가 혹시나 target을 스케일링이나 했으면 저거랑 얼추 비슷한 값이 안나오기 때문에 그걸로 스케일링을 잘못했는지 볼 수 있음!

958.7569927669903


In [85]:
# LightGBM - 워닝(FutureWarning)은 무시해도 됩니다.
import lightgbm as lgb
lg = lgb.LGBMRegressor()
lg.fit(X_tr, y_tr)
pred = lg.predict(X_val)
print(mean_absolute_error(y_val, pred))
# 952.7925407798712



[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000227 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 116
[LightGBM] [Info] Number of data points in the train set: 3292, number of used features: 17
[LightGBM] [Info] Start training from score 2562.506563
952.7925407798712


In [53]:
# 최종 제출 파일
#랜덤포레스트 모델걸로 제출

pred = rf.predict(test)
result = pd.DataFrame({
                        'pred' : pred
})
result.to_csv("result.csv", index= False)


In [57]:
# 1. pred 행의 수
print(pred.shape, result.shape)



(1764,) (1764, 1)


In [60]:
# 2. 생성한 csv 확인

#test의 행의 수와 이것이 일치하는 지봐야함
# 만약 일치 하지 않는다면 마지막에 test를 안한 경우가 있어서 개수를 다르게 제출하는 경우가 있음!!

print(pd.read_csv('result.csv').head())


        pred
0  3739.6121
1   814.1174
2  4390.1500
3   780.0642
4  1445.0092
